# 29회 ADP 복원 

In [21]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from scipy.stats import norm


import warnings
warnings.filterwarnings('ignore')

# 기계학습 (60점)

## 데이터 설명
- 데이터 출처 : https://www.data.go.kr/data/15094266/fileData.do 후처리
- 데이터 링크 : https://raw.githubusercontent.com/Datamanim/datarepo/main/adp/29/p1.csv
- 데이터 설명 : 대구도시공사_빅데이터_영구임대아파트 입주자 퇴거여부 데이터, 고유번호를 가진 계약자와 특정 아파트에 대해 매년 퇴거여부를 기록한 데이터

In [3]:
#encoding = ['utf-8','cp949','utf-8-sig','euc-kr','latin']
df =pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/adp/29/p1.csv',encoding= 'cp949')
df.head()

,순번,계약구분,재계약횟수,거주개월,아파트 이름,아파트 ID,아파트 평점,호실고유번호,층,평형대,계약자고유번호,계약서고유번호,입주연도,퇴거연도,거주연도,월세(원),보증금(원),대표나이,나이,성별,결혼여부,거주자 수,퇴거여부
0,1,유효,10,222,강남아파트,5,7.0,14520,1,12,15468,15865,2003,NaN,2008,47100,3646000,46,33,남,미혼,3,미퇴거
1,1,유효,10,222,강남아파트,5,7.0,14520,1,12,15468,15865,2003,NaN,2009,56500,4375000,46,34,남,미혼,3,미퇴거
2,1,유효,10,222,강남아파트,5,7.0,14520,1,12,15468,15865,2003,NaN,2010,56500,4375000,46,35,남,미혼,3,미퇴거
3,1,유효,10,222,강남아파트,5,7.0,14520,1,12,15468,15865,2003,NaN,2011,69900,5408000,46,36,남,미혼,3,미퇴거
4,1,유효,10,222,강남아파트,5,7.0,14520,1,12,15468,15865,2003,NaN,2012,69900,5408000,46,37,남,미혼,3,미퇴거


## 1-1  계약자고유번호를 기준으로 거주연도 별 여러개의 데이터가 쌓여 있다. 각 계약자고유번호에 대해 가장 최신의 거주연도 행만 남겨라.

In [ ]:
aa

## 1-2 결측치 처리

## 1-3 이상치 처리

## 2-1 재계약 횟수의 중앙값을 기준으로 중앙값보다 크거나 같으면 '높음', 작으면 '낮음' 으로 재계약 횟수 이분 변수를 구성하시오.

## 2-2 차원축소의 필요성을 논하고, 필요에 따라 차원을 축소하고 불필요하다면 그 근거를 논하시오.

## 3-1 재계약 횟수 이분변수를 기준으로 세그먼트를 구분하고 각 세그먼트의 특징을 분석하시오.

## 3-2. 재계약횟수 변수를 종속변수로 하는 회귀 분석을 두 가지 이상의 방법론을 통해 수행하고 최종 모델을 결정하시오. 재계약횟수 이분변수를 종속변수로 하는 분류 분석을 두가지 이상의 방법론을 통해 수행하고 최종 모델을 결정하시오. 

## 3-3 최종 채택한 모델에서 각각 유의하게 작용하는 변수를 확인 하고 설명하시오

## 3-4 해당 데이터 분석결과로 얻을 수 있는 점 제시

## 데이터 설명
- 데이터 출처 : https://www.kaggle.com/datasets/pschale/mlb-pitch-data-20152018 데이터를 후처리
- 데이터 링크 : https://raw.githubusercontent.com/Datamanim/datarepo/main/adp/29/p2_.csv
- 데이터 설명 : 

A 야구구단의 시합 결과중 일부를 나타낸다.    
각 행은 하나의 경기를 의미(game_id)하며 9회차(= 9이닝) 동안 1번타자, 2번타자의 출루 정보를 의미한다.     
각 컬럼에 대한 설명은 아래 이미지와 같으며, value값의 index는 이미지의 '수치 의미' 열을 통해 확인 할 수 있다.


<img src="https://github.com/Datamanim/datarepo/blob/main/adp/29/p1_img.png?raw=true" width="500" height="600">

## 4-1 각 회차별로 1번 타자의 출루 (1,2,3루타와 사사구(볼넷, 몸에맞는공))가 있는 경우에 대해 득점이 발생 했는지 확인하고자 한다. 이를 위한 전처리를 수행하라. (단, 첫 번째 혹은 두 번째 타자가 홈런을 친 경우 해당 회차 데이터는 제외한다.)     

조건1 : 득점여부를 범주형 종속변수로 한다. (1점이상 득점 :1, 무득점 :0)     
조건2 : 각 회차 2번 타자의 데이터는 원핫 인코딩한다.    
조건3 : 학습에 적절하지 않은 데이터는 제외한다.    

제가 이해하고 의도한 29회 4-1번 문제에 대한 가이드 입니다.
(4-1) 전처리 가이드
1. 첫타자와 둘째타자의 각 이닝별 출루현황, 해당 이닝의 득점수를 3columns의 데이터로 변환
2. 정수형 값이 아닌 데이터 행를 제거
3. 홈런 (4)값을 가지는 행을 제거
4. 이닝별 점수를 binary로 변환 (득점 있으면 1 ,없으면 0 )
5. 둘째 타자 출루 결과를 one-hot encoding

## 4-2 4-1 데이터에 대해 Logistic Regression을 적용하고 2번타자의 희생번트 여부에 대한 회귀 계수 검정을 하라

## 4-3 SMOTE (random_state =0 지정)를 적용하여 data imbalance를 해결하라

## 4-4 4-3 구성 데이터에 Logistic Regression을 적용하고 결과를 분석하라

## 4-5 4-3 구성 데이터에 XGB 적용하고 결과를 분석하라

# 통계 (40점)

## 5. 제품 A의 불량률은 0.03이다. 25개의 제품을 뽑았을 때 3개가 불량일 확률을 구하시오. (소수점 다섯 째 자리에서 반올림)

In [2]:
from scipy.stats import binom
print('25개의 제품을 뽑았을 때 3개가 불량일 확률은 {:5f}이다'.
      format(binom.pmf(3, 25, 0.03)))

25개의 제품을 뽑았을 때 3개가 불량일 확률은 0.031774이다


## 6. C사 생산 제품 1000개 중 양품이 600개, D사 생산 제품 500개 중 양품이 200개 이다. 두 회사의 양품률에 차이가 있는지 검정하여라.

In [15]:
p1 = 0.6
p2 = 0.4
n1 = 1000
n2 = 500

conf_a = 0.05
test_a = 0.05

d0 = 0
d - p1-p2
pe = (n1*p1+ n2*p2) / (n1+n2) # pooled esimate
SE2 = np.sqrt( pe*(1-pe)*(1/n1+1/n2)) # 합동 분산을 사용한 SE
zstat = d / SE2 # 검정통계량

conf_z = norm.ppf(1-conf_a/2)
ME2 = conf_z * SE2 # 오차의 한계

sp = (1-norm.cdf(np.abs(zstat)))*2
cv = norm.ppf(1-test_a/2)
print('임계값: {}, 검정통계량: {}'.format(cv, zstat))
print('유의수준: {:.3f}, 유의확률: {:.5f}'.format(test_a, sp))
print('신뢰구간: {:.3f}~{:.3f}, 검정하고자 하는 값: {:.3f}'.
     format(d-ME2, d+ME2, d0))

임계값: 1.959963984540054, 검정통계량: 7.319250547113997
유의수준: 0.050, 유의확률: 0.00000
신뢰구간: 0.146~0.254, 검정하고자 하는 값: 0.000


#### [답안] 18분
- 귀무가설 : 모비율의 차는 0이다.
- 대립가설 : 모비율의 차는 0이 아니다.
- 유의수준이 유의확률(0.05)보다 작으므로 귀무가설을 기각한다. 모비율의 차는 0이 아니다. 두 회사의 양품률에는 차이가 있다.

## 7. 아래 데이터는 a,b,c,d 네 차종 각각 5회 실험 시 범퍼 파손 정도 이다. (단, 각 모집단은 정규분포를 따르며 모집단 간 등분산성을 가정한다.)

dataurl : https://raw.githubusercontent.com/Datamanim/datarepo/main/adp/29/p7.csv

### 7-1. 각 차종 별 범퍼 파손의 정도에 차이가 유의한지 검정하라.

In [24]:
import pandas as pd
df= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/adp/29/p7.csv')
stats.f_oneway(df.loc[df.name=='A','ratio'], df.loc[df.name=='B','ratio'],
               df.loc[df.name=='C','ratio'], df.loc[df.name=='D','ratio'])

F_onewayResult(statistic=24.97695307518529, pvalue=2.8174779556216382e-06)

#### [답변]
- 귀무가설 : 각 차종별 범퍼 파손의 정도에 차이가 없다.
- 연구가설 : 적어도 하나의 차종의 범퍼 파손의 정도에 차이가 있다.
- 통계량 24.977, 유의확률(pvalue) 2.817로 유의수준 0.05보다 크므로 귀무가설을 채택한다.

### 7-2 귀무가설을 채택한다면 그 의미를 해석하고, 귀무가설을 기각하였다면 사후분석을 시행하라.

#### [답변]
- 귀무가설을 채택하는데, 각 차종별 범퍼 파손의 정도에 차이가 없다는 것을 의미한다.

## 8. L1,L2,L3 세 개의 생산라인에서 각각 13%, 37%, 50%를 생산하며 각각 1.1% , 2.1%, 3.3% 불량률을 갖는다. 불량 제품이 나왔을 때 L1 라인에서 생산되었을 확률을 구하시오. (소수점 둘째자리에서 반올림)

In [27]:
# P(L1|불량) = P(L1∩불량) / P(불량)

# P(L1) = 0.13
# P(L2) = 0.37
# P(L3) = 0.5

# P(불량|L1) = 0.011
# P(불량|L2) = 0.021
# P(불량|L3) = 0.033

# P(불량) = P(L1∩불량)      + P(L2∩불량)      + P(L3∩불량) 
#         = P(불량|L1)*P(L1) + P(불량|L2)*P(L2) + P(불량|L3)*P(L3)
L1_불량 = 0.011*0.13
불량 = 0.011*0.13 + 0.021*0.37 + 0.033*0.5

print('[답안]')
print('불량 제품이 나왔을 때 L1 라인에서 생산되었을 확률은 {:.3f}입니다.'.
      format(L1_불량/불량))

[답안]
불량 제품이 나왔을 때 L1 라인에서 생산되었을 확률은 0.056입니다.


- (24분 소요)

created by datamanim [web-link](https://www.datamanim.com)

#### 끝. 10페이지